In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import mode
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
filtered_profile = pd.read_csv('/Users/jennytang/Desktop/2rb/data/saved/filtered_profile.csv')
filtered_profile #152057 rows × 40 columns

# feature engineering: 
- convert variables to meaningful info : birthday(age) and created_at_x(account length)
- fill missing values 
- dummify categorical values


### - convert variables to meaningful info : birthday(age) and created_at_x(account length)

In [ ]:
# birthday(age)
this_year = 2017
filtered_profile['birthday'] = pd.to_datetime(filtered_profile['birthday']).dt.year
filtered_profile['age'] = this_year - filtered_profile['birthday']
filtered_profile['age']

In [ ]:
# created_at_x(account_length)
today = pd.to_datetime('2017-12-31')
filtered_profile['created_at_x'] = pd.to_datetime(filtered_profile['created_at_x'])
filtered_profile['account_length'] = (today - filtered_profile['created_at_x']).dt.days
filtered_profile['account_length']

In [ ]:
filtered_profile = filtered_profile.drop(['birthday','created_at_x'],axis = 1)
print filtered_profile.columns

In [ ]:
filtered_profile.shape

##### making flags for missing values

In [ ]:
for col in filtered_profile.columns:
    filtered_profile[col+"_missing"] = filtered_profile[col].isnull()
filtered_profile

In [ ]:
len(filtered_profile.columns.tolist()) # for each col, there's a correspongding col_missing flag col

### For more accurate modeling, drop some special cases

In [ ]:
len(filtered_profile[filtered_profile['sex'] == filtered_profile['seeking']]) 

In [ ]:
special_case = filtered_profile[filtered_profile['sex'] == filtered_profile['seeking']]
special_case

In [ ]:
filtered_profile = filtered_profile.drop(special_case.index)
filtered_profile.shape

### Fill missing values 

In [ ]:
filtered_profile.info() # check which columns are missing values

### missing location(lat and lng)
- use the users who receive their messages's average location to fill missing location
- if they message the person that means the message receiver might be close to the user's location so we can use that to guess his location

In [ ]:
len(filtered_profile[filtered_profile['lat'].isnull()==True]),len(filtered_profile[filtered_profile['lng'].isnull()==True])

In [ ]:
msk = filtered_profile[filtered_profile['lat'].isnull()==True]
filtered_profile = filtered_profile.drop(msk.index,axis =0)

In [ ]:
filtered_profile

In [ ]:
# initial_message = pd.read_csv('path/first_message_list.txt')

In [ ]:
# missing_location_user_id = filtered_profile['user_id'][filtered_profile['lat'].isnull()==True]

In [ ]:
# user_miss_loc_message = initial_message[initial_message['sender_id'].isin(missing_location_user_id)]
# print 'number of unique users who didnt fill their location but messaged other people show interest:', len(user_miss_loc_message['sender_id'].unique())

In [ ]:
# user_miss_loc_message.columns

In [ ]:
# temp = user_miss_loc_message
# msk = filtered_profile['user_id'].isin(user_miss_loc_message['receiver_id'])
# len(filtered_profile[msk]['lat'])
# len(temp)
# # temp['lat']=filtered_profile.loc[msk]['lat']
# # temp.loc[msk,'lat'] = filtered_profile[msk]['lat']
# # temp.loc[msk,'lng'] = filtered_profile[msk]['lng']

### missing zodiac and cn_zodiac
fill out both missing zodiac and cn_zodiac in 0 since the defined range is from 1-12
- ZODIAC ={ 
    "capricorn" => 1,
    "aquarius" => 2,
    "pisces" => 3,
    "aries" => 4,
    "taurus" => 5,
    "gemini" => 6,
    "cancer" => 7,
    "leo" => 8,
    "virgo" => 9,
    "libra" => 10,
    "scorpio" => 11,
    "sagittarius" => 12}

- CHINESE_ZODIAC = {
    "rat" => 1,
    "ox" => 2,
    "tiger" => 3,
    "rabbit" => 4,
    "dragon" => 5,
    "snake" => 6,
    "horse" => 7,
    "sheep" => 8,
    "monkey" => 9,
    "rooster" => 10,
    "dog" => 11,
    "pig" => 12}

In [ ]:
filtered_profile['zodiac'].value_counts() #categorical #fill in 0
filtered_profile['zodiac'] = filtered_profile['zodiac'].fillna(0)
filtered_profile['zodiac']

In [ ]:
filtered_profile['cn_zodiac'].value_counts() #categorical #fill in 0
filtered_profile['cn_zodiac'] = filtered_profile['cn_zodiac'].fillna(0)
filtered_profile['cn_zodiac']

### missing age
fill ages with mode separeated by genders: numerical
- based on sex, use mode of each gender group
- => f_mode_age: 27 and m_mode_age: 27

In [ ]:
filtered_profile['age'].fillna(0,inplace=True)
filtered_profile['age'].isnull().value_counts()

valid_age_condition = filtered_profile['age'] >= 16 # include 0(nan) and negative numbers

invalid_conditionf = (~valid_age_condition) & (filtered_profile['sex']==2)
# print len(filtered_profile[invalid_conditionf])

invalid_conditionm = (~valid_age_condition) & (filtered_profile['sex']==1)
# print len(filtered_profile[invalid_conditionm])

temp = filtered_profile[valid_age_condition] #valid ages
print 'Number of invalid ages:', len(filtered_profile)-len(temp) # number of valid ages and nan = 26369


f_mode_age = mode(temp[temp['sex']==2]['age']).mode[0]
print 'within valid female ages, the mode age is:', f_mode_age
m_mode_age = mode(temp['age'][temp['sex']==1]).mode[0]
print 'within valid male ages, male mode age is:', m_mode_age
# print len(filtered_profile['age'])

filtered_profile.loc[invalid_conditionf,'age'] = f_mode_age
filtered_profile.loc[invalid_conditionm,'age'] = m_mode_age

filtered_profile['age'].isnull().value_counts()

### missing education
fill missing education: can be filled in mean or mode based on sex  
- EDUCATION ={"high_school" => 1 , "some_college" => 2, "bachelor" =>3, "master" => 4, "phd" => 5, "associate" => 6}
- => both mean and mode for both f and m turn out are 3!!! -> "bachelor" 


In [ ]:
filtered_profile[filtered_profile['education'].isnull()==True]

In [ ]:
print filtered_profile['education'].value_counts()  #categorical  #0s means not filled :

#can be filled in mean / mode based on sex 
temp2 = filtered_profile[(filtered_profile['education'].isnull() == False) & (filtered_profile['education']!=0) ] # ppl who filled edu
print 'Number of missing edu:', len(filtered_profile)-len(temp2) # number of missing edu
print len(temp2)
print
f_mode_edu = mode(temp2[temp2['sex']==2]['education']).mode[0]
print 'mode edu of female is:', f_mode_edu
m_mode_edu = mode(temp2[temp2['sex']==1]['education']).mode[0]
print 'mode edu of male is:', m_mode_edu

f_mean_edu = temp2[temp2['sex']==2]['education'].mean()
print 'mean edu of female is:', f_mean_edu
m_mean_edu = temp2[temp2['sex']==1]['education'].mean()
print 'mean edu of male is:', m_mean_edu
# mode and mean for both f and m are 3! 

mask = (filtered_profile['education'].isnull() == True) | (filtered_profile['education']==0)
filtered_profile.loc[mask,'education'] = f_mode_edu 

filtered_profile['education'].value_counts()

### missing birth place
fill birth place by mode
- BIRTH_PLACE ={
    "United States" => 1,
    "Canada" => 2,
    "China" => 3,
    "HongKong" => 4,
    "Taiwan" => 5,
    "Macao" => 6,
- => top 3: United states, Macao, Canda
- fill in mode: US 

In [ ]:
#can be filled in mode
temp3 = filtered_profile[(filtered_profile['birth_place'].isnull() == False) & (filtered_profile['birth_place']!=0) ] # ppl who filled 'birth_place'
print '% of user birthplaces among the users who filled : '
print temp3['birth_place'].value_counts()/len(temp3) # categorical  #0s means not filled 
print 
print 'Number of missing birth_place:', len(filtered_profile)-len(temp3) # number of missing edu
print
mode_birth_place = mode(temp3['birth_place']).mode[0]
print 'most freq birth_place is:', mode_birth_place

mask = (filtered_profile['birth_place'].isnull() == True) | (filtered_profile['birth_place']==0)

filtered_profile.loc[mask,'birth_place']= mode_birth_place
print 
print '% of user birthplaces among all users after filling the mode:' 
print filtered_profile['birth_place'].value_counts()/len(filtered_profile['birth_place'])
# fill in mode chnaged % of ppl birth place in U.S. increased by 0.15

### missing ethnicity
fill ethnicity
-   ETHNICITY ={
    "chinese" => 1,
    "cambodian" => 2,
    "fillipino" => 3,
    "japanese" => 4,
    "korean" => 5,
    "laotian" => 6,
    "malaysian" => 7,
    "pacific_islander" => 8,
    "tai" => 9,
    "vietnamese" => 10,
    "other_asian" => 11,
    "black" => 12,
    "hispanic" => 13,
    "indian" => 14,
    "middle_eastern" => 15,
    "native_american" => 16,
    "white_caucasian" => 17,
    "other" => 18}
- Chinese among all filled Ethnicity are  0.944345
- made for both f and m are chinese! fill in mode 1 as chinese for all

In [ ]:
print filtered_profile['ethnicity'].value_counts() #categorical  #0s means not filled 

temp4 = filtered_profile[(filtered_profile['ethnicity'].isnull() == False) & (filtered_profile['ethnicity']!=0) ] # ppl who filled 'Ethnicity'
print "% of user enthnicity among the users who filled"
print temp4['ethnicity'].value_counts()/len(temp4['ethnicity'])
# 1.0 Chinese are  0.944345!!!! 

f_mode_eth = mode(temp4[temp4['sex']==2]['ethnicity']).mode[0]
print 'mode ethnicity of female is:', f_mode_eth
m_mode_eth = mode(temp4[temp4['sex']==1]['ethnicity']).mode[0]
print 'mode ethnicity of male is:', m_mode_eth

mask = (filtered_profile['ethnicity'].isnull() == True) | (filtered_profile['ethnicity']==0)
filtered_profile.loc[mask,'ethnicity'] = m_mode_eth
filtered_profile['ethnicity'].value_counts()/len(filtered_profile['ethnicity'])

### missing body type
fill body type by gender
-   BODY_TYPE = {"Slender" => 1, "Athletic" =>2, "Average build" =>3, "Curvy" =>4}
-  most male and female think they are average build 

In [ ]:
print filtered_profile['body_type'].value_counts() #categorical  #0s means not filled 

temp5 = filtered_profile[(filtered_profile['body_type'].isnull() == False) & (filtered_profile['body_type']!=0) ] # ppl who filled 'body_type'

f_mode_body = mode(temp5[temp5['sex']==2]['body_type']).mode[0]
print 'mode boday type of female is:', f_mode_body
m_mode_body = mode(temp5[temp5['sex']==1]['body_type']).mode[0]
print 'mode body type of male is:', m_mode_body

mask = (filtered_profile['body_type'].isnull() == True) | (filtered_profile['body_type']==0)
filtered_profile.loc[(mask),'body_type'] = f_mode_body
filtered_profile['body_type'].value_counts()

### missing have_children
fill have_children
- has_children ={"No" => 1 , "Yes" => 2}
- => only 0.293952 female users with kid and 0.198786 male users with kids
- fill in 1 as no for missing data


In [ ]:
filtered_profile['have_children'].value_counts() #categorical  #0s means not filled 

temp6 = filtered_profile[(filtered_profile['have_children'].isnull() == False) & (filtered_profile['have_children']!=0) ] # ppl who filled 'have_children

f_mode_child = mode(temp6[temp6['sex']==2]['have_children']).mode[0]
print 'mode of have child of female is:', f_mode_child
m_mode_child = mode(temp6[temp6['sex']==1]['have_children']).mode[0]
print 'mode of have child of male is:', m_mode_child
print 

percent_f_child = (temp6[temp6['sex']==2]['have_children']).value_counts()/len(temp6[temp6['sex']==2])
print '% of female have children is:'
print percent_f_child
print
percent_m_child = (temp6[temp6['sex']==1]['have_children']).value_counts()/len(temp6[temp6['sex']==1])
print '% of male have children is:'
print percent_m_child
# both f and m are 'no'


mask = (filtered_profile['have_children'].isnull() == True) | (filtered_profile['have_children']==0)
filtered_profile.loc[(mask),'have_children'] = f_mode_child
print 
print '% of users have kids:'
print filtered_profile['have_children'].value_counts()/len(filtered_profile['have_children'])



### missing marital status
fill marital status
- MARITAL_STATUS ={"Never_married" => 1 , "Divorced" => 2, "Widowed" => 3, "Seperated"=>4}
- => 0.78 female users are never married;  0.87 of male users are never married
- fill in mode 1 as never married 

In [ ]:
print filtered_profile['marital_status'].value_counts()
drop_rows = filtered_profile[filtered_profile['marital_status'] == 10]
filtered_profile = filtered_profile.drop(drop_rows.index)
filtered_profile

In [ ]:
temp7 = filtered_profile[(filtered_profile['marital_status'].isnull() == False) & (filtered_profile['marital_status']!=0) ] # ppl who filled 'marital_status'

f_mode_marital = mode(temp7[temp7['sex']==2]['marital_status']).mode[0]
print 'mode of marital_status of female is:', f_mode_marital
m_mode_marital = mode(temp7[temp7['sex']==1]['marital_status']).mode[0]
print 'mode of marital_status of male is:', m_mode_marital
print 

percent_f_marital = (temp7[temp7['sex']==2]['marital_status']).value_counts()/len(temp7[temp7['sex']==2])
print '% of female marital_status among female who filled:'
print percent_f_marital
print
percent_m_marital = (temp7[temp7['sex']==1]['marital_status']).value_counts()/len(temp7[temp7['sex']==1])
print '% of male marital_status among male who filled:'
print percent_m_marital
# both f and m are 'never married'

mask = (filtered_profile['marital_status'].isnull() == True) | (filtered_profile['marital_status']==0)
filtered_profile.loc[(mask),'marital_status'] = f_mode_marital
print 
print '% of all users marital status after fill in mode:'
print filtered_profile['marital_status'].value_counts()/len(filtered_profile['marital_status'])


### missing occupation
fill occupation
- OCCUPATION = {"human_resources" => 1, "artistic" => 2, "education" => 3, "engineering" => 4, "accounting" => 5, "executive" => 6, "law" => 7, "medical" => 8, "military" => 9, "political" => 10, "retail" => 11, "retired" => 12, "sales" => 13, "student" => 14, "other" => 15}

In [ ]:
filtered_profile['occupation'].value_counts() #categorical  #0s means not filled 


temp8 = filtered_profile[(filtered_profile['occupation'].isnull() == False) & (filtered_profile['occupation']!=0) ] # ppl who filled 'marital_status'

f_mode_occupation = mode(temp8[temp8['sex']==2]['occupation']).mode[0]
print 'mode of occupation of female is:', f_mode_occupation
m_mode_occupation = mode(temp8[temp8['sex']==1]['occupation']).mode[0]
print 'mode of occupation of male is:', m_mode_occupation
print 

percent_f_occupation = (temp8[temp8['sex']==2]['occupation']).value_counts()/len(temp8[temp8['sex']==2])
print '% of female occupation among female who filled:'
print percent_f_occupation
plt.title('percent_female_user_occupation')
percent_f_occupation.plot(kind='bar')
plt.show()

percent_m_occupation = (temp8[temp8['sex']==1]['occupation']).value_counts()/len(temp8[temp8['sex']==1])
print '% of male occupation among male who filled:'
print percent_m_occupation
plt.title('percent_male_user_occupation')
percent_m_occupation.plot(kind='bar')
plt.show()

In [ ]:
mask = (filtered_profile['occupation'].isnull() == True) | (filtered_profile['occupation']==0)
filtered_profile.loc[(mask)&(filtered_profile['sex']==2),'occupation'] = 5
filtered_profile.loc[(mask)&(filtered_profile['sex']==1),'occupation'] = 4

filtered_profile['occupation'].value_counts()/len(filtered_profile['occupation'])

In [ ]:
filtered_profile.info()

### saved fill missing data to local 'filtered_profile_fill'

In [ ]:
filtered_profile.columns

In [ ]:
# need to drop other 5 cols => 33 features left
filtered_profile.drop(second_drop_cols, axis=1, inplace=True)

In [ ]:
# need to drop missing_cols which are have completed data 
filtered_profile.drop(third_drop_cols, axis=1, inplace=True)

In [ ]:
filtered_profile.columns.tolist()

In [ ]:
filtered_profile.to_csv('path/filtered_profile_fill.csv',sep=',')

### read save file as 'filtered_profile_fill'

In [ ]:
filtered_profile_fill = pd.read_csv('path/filtered_profile_fill.csv')
filtered_profile_fill # 144586 rows × 44 columns

In [ ]:
filtered_profile_fill.columns.tolist()

### - dummify categorical values

In [ ]:
id_cols = ['user_id'] #1

categ_cols = ['categ_cols in system'] #15

numeric_cols = ['numeric_cols in system'] #16
missing_cols = set(filtered_profile_fill.columns.tolist())- \
set(id_cols)- \
set(categ_cols)- \
set(numeric_cols) 

len(id_cols),len(categ_cols),len(numeric_cols),len(missing_cols)

In [ ]:
missing_cols #10

In [ ]:
filtered_profile_fill = pd.get_dummies(filtered_profile_fill, 
                                       columns= categ_cols) 

In [ ]:
filtered_profile_fill.columns.tolist() #144586 rows × 128 columns

In [ ]:
filtered_profile_fill

#### drop system accounts: 2 rows

In [ ]:
system_account = [1,2,3,4]
drop_system_account = filtered_profile_fill[filtered_profile_fill['user_id'].isin(system_account)]
drop_system_account

In [ ]:
filtered_profile_fill.drop(drop_system_account.index, inplace=True)

In [ ]:
filtered_profile_fill

### saved dummified profile

In [ ]:
filtered_profile_fill.to_csv('path/filtered_profile_fill_dum.csv')

### read dummified profile below 

In [ ]:
filtered_profile_fill_dum = pd.read_csv('path/filtered_profile_fill_dum.csv')
filtered_profile_fill_dum #144584 rows × 128 columns


### The final preprocessed file is filled with missing values and dummies